# Imports

In [1]:
import pandas as pd
from typing import Iterator, Tuple
from pyspark.sql import SparkSession, Window
import pyspark.sql.functions as F
from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import *

In [2]:
spark = (SparkSession
         .builder
         .appName('Chapter 7')
         .enableHiveSupport()
         .getOrCreate())
spark

# Spark Configurations 

We can also use the __Environment__ tab in the UI to check for all the configurations (they are read-only).

We can set the configurations:
- Using the conf files in the Spark dir
- Using command line with spark-submit
- In the Spark application itself

In [3]:
spark.sql('SET -v').count()

119

In [4]:
spark.sql('SET -v').select('key', 'value').show(truncate=False)

+-----------------------------------------------------------+----------------------------------------------------------------+
|key                                                        |value                                                           |
+-----------------------------------------------------------+----------------------------------------------------------------+
|spark.sql.adaptive.advisoryPartitionSizeInBytes            |<value of spark.sql.adaptive.shuffle.targetPostShuffleInputSize>|
|spark.sql.adaptive.coalescePartitions.enabled              |true                                                            |
|spark.sql.adaptive.coalescePartitions.initialPartitionNum  |<undefined>                                                     |
|spark.sql.adaptive.coalescePartitions.minPartitionNum      |<undefined>                                                     |
|spark.sql.adaptive.enabled                                 |false                                             

In [5]:
spark.conf.get('spark.sql.shuffle.partitions')

'200'

In [6]:
spark.conf.set('spark.sql.shuffle.partitions', 5)
spark.conf.get('spark.sql.shuffle.partitions')

'5'

Among all the ways that you can set Spark properties, an order of precedence determines which values are honored. Any values or flags defined in spark-defaults.conf will be read first, followed by those supplied on the command line with spark- submit, and finally those set via SparkSession in the Spark application. All these properties will be merged, with any duplicate properties reset in the Spark application taking precedence. Likewise, values supplied on the command line will supersede settings in the configuration file, provided they are not overwritten in the application itself.

# Scaling Spark for Large Workloads

## Resource Allocation

The main three arguments that cover the resources allocation in a Spark application are:
- `--num_executors`
- `--executor-cores`
- `--executor-memory`

As a background:
- An executor is a __JVM__. So one executor can run multiple tasks at the same time using JVM __threads__ where each thread will have its own CPU core.
- Driver program has the SparkContext, schedules, and task.
- Cluster manager allocate resource in the form of containers. For example, __YARN__ has:
    - __Master Node__ that has __Resource Manager__, which consists of __Scheduler__ and __Application Manager__
    - __Application Master__ that sits on one of the worker nodes
    - __Node Manager__ (for each worker node) that manages the status of computations of the containers
    
Therefore, we need to have some resources to the cluster manager and its daemons to run.

Let's assume we have a cluster of 4 nodes, 16 cores/node, 64 GB of memory/node. What is the optimal allocation:
1. __Most granular:__ 
    - __1 executor = 1 core__. This is not good because each executor will run only one task at a time and we can't benefit from starting and initializing the executor (which is an overhead) because we can't run multiple tasks at the same time since each executor has only __1 core__
2. __Least granular:__
    - 1 executor = 16 cores. Even though this achieves the max parallelism, it isn't is not good as well because we need to have resources for __Node Managers__ and other Hadoop auxiliary daemons to run on each node.
3. __Best config:__
    - For each node, leave 1 core & 1 GB for Node Manager and other daemons. Therefore, we would be left with 15 cores and 63 GB of memory on each node.
    - Since we need resources to launch AppMaster and we also have issues with HDFS throughput, it is recommended to have 5 cores per executor.
    - Therefore, the optimal configuration is as follows:
        - 1 core and 1 GB per node for Node Manager and other auxiliary daemons
        - 5 cores per executor to maximize HDFS throughputs
        - 60 / 5 = 12 executors.
            - 1 container for AppMaster. This means we end up with 11 executors.
        - 63 GB / 3 executors = 21 GB of memory for each node.
            - 10% of the memory is typically allocated for off-heap (mainly used for direct buffers). Therefore, we have 21 * 0.9 = 19 GB.

## Static vs Dynamic Resource Allocation

If we cap the limit of resources Spark applications can use:
- May acquire a lot more resources (executors) than it needs
- May fail because of performance degradation since it received much more workloads than anticipated and can't get more resources

Therefore, it is better to have dynamic resource allocation so that the application would demand more resources if needed and free resources if not needed. __Note__ that this may affect other applications that are running at the same time if we have large workloads on the same cluster.

```bash
spark.dynamicAllocation.enabled true
spark.dynamicAllocation.minExecutors 2
spark.dynamicAllocation.schedulerBacklogTimeout 1m
spark.dynamicAllocation.maxExecutors 20
spark.dynamicAllocation.executorIdleTimeout 2min
```

1. By default, dynamic allocation is set to false. When enabled, Spark driver would ask the cluster manager for more executors if needed and free executers if not needed.
2. The Spark driver would start the application with 2 executors (`spark.dynamicAllocation.minExecutors 2`).
3. If the backlog timeout (1 minute) where we have more tasks in the queue that have not been scheduled yet, then Spark driver asks the cluster manager for more executors each time the backlog timeout.
    - `spark.dynamicAllocation.sustainedSchedulerBacklogTimeout` define when to check if there are still pending tasks to ask for more executors.
    - The number of executors requested in each round increase exponentially; for example, start with 1, 2, 4, 8.
4. The max number of executors that Spark driver can acquire for the application is 20 (`spark.dynamicAllocation.maxExecutors 20`)
5. If an executor finished a task and is idle for more than 2 minutes (`spark.dynamicAllocation.executorIdleTimeout 2min`) -> the driver will terminate it. Therefore, free that executor.

- By default, Spark uses long-running containers for speed. This means that we have underutilized resources since some of those containers maybe idle.
-  `spark.dynamicAllocation.cachedExecutorIdleTimeout = infinity` is the default, that means executors with cached data won't be terminated by Spark. We may need to set it to some value to better utilize the cluster's resources.

## Executor's Memory

The executor's memory is controlled by `spark.executor.memory`. There are 3 sections of executor memory:
- Execution memory
- Storage memory
- Reserved memory

The default is 60% for execution memory and 40% for storage memory. If part of storage memory is not used and the executor need more memory -> acquire memory from storage memory and vice versa. 300 MB will be reserved memory to avoid Out-Of-Memory errors. 

Execution memory is used for Spark shuffles, joins, sorts, and aggregations. Since different queries may require different amounts of memory, the fraction (`spark.memory.fraction` is 0.6 by default) of the available memory to dedicate to this can be tricky to tune but it’s easy to adjust. By contrast, storage memory is primarily used for caching user data structures and partitions derived from DataFrames. During map and shuffle operations, Spark reads and writes to local disk's shuffle files -> heavy I/O operations.

If storage is full, Spark uses LRU strategy to evict blocks. __It is better to evict from storage memory (not execution memory)__ because we most likely will need the data we evicted soon from the execution memory to be used in computation. However, we may not need the data evicted from the storage memory.

`spark.memory.storageFraction` is the threshold that determines the fraction of the storage memory that are immune to eviction. The default is 50%.

## Maximizing Parallelism

Within each job stage, there will be many tasks. Spark will at best schedule a thread per task per core, and each task will process a distinct partition. To optimize resource utilization and maximize parallelism, the ideal is at least as many partitions as there are cores on the executor. If there are more partitions than there are cores on each executor, all the cores are kept busy; a single thread running on a single core can work on a single partition.

__Partion__ is a subset of data (contiguous block of data on disk). Data is laid out on disk as contiguous block file blocks where blocks are typically 64 MB or 128 MB. HDFS's default block size is 128 MB which is also the default size of the partition. It can be controlled with `spark.sql.files.maxPartitionBytes`.

__Shuffle Partitions__ are created during the shuffle stage. The default value is 200 partitions, which is too large for small datasets or streaming workloads because this will cause sending small partitions across the network which is very expensive (latency). We can change this value to be at most the number of cores in the executors using `spark.sql.shuffle.partitions`.

During shuffling stage; which typically happens with wide transformations such as `groupby` or `join`, the shuffle will spill results to executors' local disk at `spark.local.directory`. Therefore, shuffles consume both network and disk I/O resources.

## Caching/Persisting

Caching will persist the data and the data lineage in memory. It is a good idea to cache DataFrames in memory if:
- The same data is being accessed iteratively during tasks such as training ML model
- The same data is used to do multiple transformations during ETL

However, be careful if the dataset is very large and can't fit into memory because it may require intensive use of resources during serialization/deserialization.

In [10]:
from pyspark import StorageLevel

In [9]:
df = spark.range(1 * 10000000).toDF("id").withColumn("square", F.col("id") * F.col("id"))
df.show(5)

+---+------+
| id|square|
+---+------+
|  0|     0|
|  1|     1|
|  2|     4|
|  3|     9|
|  4|    16|
+---+------+
only showing top 5 rows



In [24]:
df.cache().count()

10000000

In [25]:
df2 = spark.range(1 * 10000000).toDF("id").withColumn("square", F.col("id") * F.col("id"))
df2.cache().count()

10000000

In [26]:
df.explain()

== Physical Plan ==
*(1) Project [id#80L, (id#80L * id#80L) AS square#84L]
+- *(1) Range (0, 10000000, step=1, splits=2)




In [27]:
df2.explain()

== Physical Plan ==
*(1) ColumnarToRow
+- InMemoryTableScan [id#327L, square#329L]
      +- InMemoryRelation [id#327L, square#329L], StorageLevel(disk, memory, deserialized, 1 replicas)
            +- *(1) Project [id#80L, (id#80L * id#80L) AS square#84L]
               +- *(1) Range (0, 10000000, step=1, splits=2)




`df.unpersist()` will free up the memory; regardless if we used `df.cache()` or `df.persist(storageLevel)`.

## Checkpointing

- Checkpointing is reliable because it stores the data in reliable storage (__HDFS__). Therefore, we need to delete those files once done with our Spark application because they will stay on disk if we don't explicitly delete them.
- It is only available for RDDs, we can't checkpoint DataFrames because we can't save data lineage graph.
- It is slow compared to caching.

There are two modes for checkpointing: __Eager__ (performs the checkpointing right away) or __Lazy__ (wait until an action is called on the dataset). Checkpointing makes sense if the cluster is noisy; i.e. too many users are competing for resources to run computations and too many jobs are running simultaneously. This means that if one executor fails and Spark wants to recompute the results of the partitions of that executor, it may take very long time due to too many jobs running at the same time and too many users are competing for the cluster resources.

## Speculative Execution

Spark jobs run on clusters of commodity hardwares. This may lead to the situation that is called __Straggler__, which means that some tasks may run significantly slower than all other tasks.

Straggler happens due to:
1. Equal workload, unequal resources. This is the easiest. It means that some nodes are much more computationally powerful than other nodes. Therefore, tasks that run on the less performant nodes will take longer to finish.
2. Equal resources, unequal workload. This happens due to __Data Skew__. It means that some executors have much bigger partitions than other executors. To solve this issue, we can repartition the data or use salting.

- `spark.speculation = false`: If set to "true", performs speculative execution of tasks. This means that after a set % of tasks finishes, Spark reschedule the pending ones. If set to false, which the default, Spark will wait until all tasks are completed and no rescheduling will take place.
- `spark.speculation.interval = 100ms`
- `spark.speculation.multiplier = 1.5`
- `spark.speculation.quantile = 0.75`: Fraction of tasks which must be complete before speculation is enabled for a particular stage

For example, if we have 100 tasks, medium execution time = 120s and spark.speculation.multiplier = 1.5 and spark.speculation.quantile = 0.75. This means that 75 tasks should finish (100 * 0.75) and a task is considered straggler > 180s (120 * 1.5). Therefore, tasks that take more than 180s will be rescheduled.

# Spark Joins

Joins by default triggers large movement of data across Spark executors.

## Broadcast Hash Join (Map-side-only join)

If we have one large dataset (ideally don't fit in memory) and a small dataset (<= 10 MB) where we want to do inner joins, the driver will broadcast the small dataset into the executors. This way the data will be available in memory for all executors and the join will take place locally -> no movement of the data across the network.

By default, Spark with do __Broadcast Hash Join__ if one of the datasets size <= 10 MB. We can control the size using `spark.sql.autoBroadcastJoinThreshold` (if we put it to __-1__, it will force Spark to use shuffle sort merge join). However, be careful to force the join to be broadcat hash join because the driver/executors' memory may not have space and will get Out-of-Memory errors.

Note that we need to be sure that each key in both datasets is hashed to the same partition by Spark.

## Shuffle Sort Merge Join

Merge the two large datasets using a common key that is sortable, unique, and can be stored in the same partition. This means all rows that have the same hash value of the common key will be in the same partition on the same executor. Therefore, data has to be exchanged between executors across the network.

The algorithm has two phases:
1. Sort phase: Sort each dataset by the desired common key.
2. Merge Phase: Iterates over each key in the row of each dataset and merges the rows if the two keys match.

We can avoid the __Exchange__ part of the job if we sort the common keys and bucket them using the common keys.

By default, it is enabled using `spark.sql.join.preferSortMerge Join`.